In [1]:
from glob import glob
import os
join = os.path.join
import nibabel as nib
import numpy as np
import torchio as tio
import SimpleITK as sitk

In [2]:
def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum

In [3]:
img_path = '/home/t722s/Desktop/Datasets/BratsTestData/imagesTs/BraTS2021_01646.nii.gz'
mask_gt_path = '/home/t722s/Desktop/Datasets/BratsTestData/labelsTs/BraTS2021_01646.nii.gz'
pred_path = '/home/t722s/Desktop/Sam-Med3DTest/segmentation_maps/BraTS2021_01646_pred2.nii.gz'

sitk_image = sitk.ReadImage(img_path)
sitk_label = sitk.ReadImage(mask_gt_path)


# Try to get the (modified) gt used for the dice comparison
infer_transform = [
        tio.ToCanonical(),
        tio.CropOrPad(mask_name='label', target_shape=(128, 128, 128)),
    ]

transform = tio.Compose(infer_transform)

if sitk_image.GetOrigin() != sitk_label.GetOrigin():
    sitk_image.SetOrigin(sitk_label.GetOrigin())
if sitk_image.GetDirection() != sitk_label.GetDirection():
    sitk_image.SetDirection(sitk_label.GetDirection())

subject = tio.Subject(
    image = tio.ScalarImage.from_sitk(sitk_image),
    label = tio.LabelMap.from_sitk(sitk_label),
)


In [4]:

subject = transform(subject)
gt3d = subject.label.data.clone()

sitk_pred = sitk.GetArrayFromImage(sitk.ReadImage(pred_path))
compute_dice(gt3d[0].numpy().astype(np.uint8), sitk_pred)

0.7659287776708373

In [5]:
subject.label

LabelMap(shape: (1, 128, 128, 128); spacing: (1.50, 1.50, 1.50); orientation: RAS+; dtype: torch.ByteTensor; memory: 2.0 MiB)